# MLFM PVLIB 211213
### Steve Ransome (SRCL) and Juergen Sutterlueti (Gantner Instruments)  
#### Tutorial overview.

I) The Loss Factors Model (LFM) |2011 ref 1| quantifies  
normalised losses from module parameters  
(e.g. i_sc, r_sc, i_mp, v_mp, r_oc and v_oc) by analysing the shape  
of the IV curve and comparing it with STC reference values from the datasheet.  

II) The Mechanistic performance model (MPM) |2017 ref 2|  
has "meaningful,independent, robust and normalised" coefficients  
which fit how the LFM values depend on irradiance, module temperature  
(and windspeed) and time.  
  
These parameters relate to  
    - c_1 = "overall performance quality"  
    - c_2 = "normalised temperature coefficient" (units /K)  
    - c_3 = "low light level drop" due to v_oc and r_sc (r_shunt)  
    - c_4 = "high light level fall" due to r_oc (~Rseries).  
    (optional)  
    - c_5 = "wind speed coefficient"  
    - c_6 = "low light level drop" sometimes needed for r_shunt behaviour.  

III) This tutorial shows how to take module measured and weather data,  
(either outdoor or IEC 61853-like   
matrix data), normalise it, generate MLFM coefficients, fit them with the MPM then  
analyse module performance looking for loss values, degradation and  
allowing performance predictions.   

Fig 1: MLFM overview flow chart of this tutorial.   
![mlfm_flow.png](mlfm_data/figs/mlfm_flow.png)  

In [ ]:
#import pvlib
from pvlib import *

# Import essential library file with lfm and mpm definitions
from mlfm import *
# Import graphics code from separate file
from mlfm_graphs import *

# STANDARD DEFINITIONS

# https://matplotlib.org/stable/tutorials/introductory/customizing.html
plt.rcParams['figure.figsize'] = [7,5]  # setup figure size inches
plt.rcParams.update({'font.size': 12})  # setup fontsize
plt.linewidth = 1.5                     # line width in points
plt.linestyle = '--' #-                 # solid line
plt.marker = 's' #o                     # the default marker
plt.markersize = 9 #6                   # marker size, in points
plt.bbox = 1.4                          # offset right to not overwrite

In [ ]:
# FIND AND SHOW INSTALLATION CURRENT WORKING DIRECTORY
import os
root_dir = os.getcwd()

root_dir

## Select MLFM measurement files

File naming conventions can be used to help identify files, for example  
`x81_T1906_D3_Fh.csv`  

where   
   - x = source e.g. (G)antner, (N)rel, (S)andia, matri(X), ...  
   - 81 = module id/channel number                               
   - T1906 = (T)ime started = yymm(dd)                      
   - D3 = (D)uration in days                             
   - Fh = (F)requency e.g. (h)ours  or (10m)10 minutes      
   - etc.  
   
   
   Three default files are included (* = version number )  
(-1) t1_041.csv   
(0) g78_T16_Xall_F10m_R900*.csv  
(1) n05667_Y13_R1k6_fClear*.csv  
(2) x19074001_iec61853*.csv      


Essential default column names in meas() are :-  

meas {  
'date_time', 'module_id', 'poa_global', 'wind_speed', 'temp_air',  
'temp_module', 'v_oc', 'i_sc', 'i_mp', 'v_mp', 'r_sc', 'r_oc'  
}


In [ ]:
# select one of the following meas files
meas_file = 0

if meas_file == 0:
    mlfm_meas_file = 'g78_T16_Xall_F10m_R900_041.csv'
elif meas_file == 1:
    mlfm_meas_file = 'n05667_Y13_R1k6_fClear_041.csv'
elif meas_file == 2:
    mlfm_meas_file = 'x19074001_iec61853_041.csv'

# optional
# elif meas_file == -1:
#    mlfm_meas_file = 't1_041.csv'
    
# extract module id from filename e.g. 'g78'
mlfm_mod = mlfm_meas_file.split('_')

mlfm_mod_sel = mlfm_mod[0]

## Get ref module data at STC  

Get STC Reference module data for the selected module  
(searching for a row with same module id).  
  
Ref values include electrical data and temperature coeffs and must include the following :-  

ref {  
'i_sc', 'i_mp', 'v_mp', 'v_oc',   
'alpha_i_sc', 'alpha_i_mp', 'beta_v_mp', 'beta_v_oc', 'gamma_p_mp', 
}  

NOTE : Users must add their own data to the reference file  
when they add new meas data. 

### Read all ref data 

In [ ]:
# user must keep updated with their modules from their measurements
ref_file_name = (root_dir + '\\mlfm_data\\ref\\mlfm_reference_modules.csv')

ref_data = pd.read_csv(ref_file_name, index_col='module_id')

### Select stc data from reference database

In [ ]:
while True:
    try:
        ref_data = ref_data[ref_data.index == mlfm_mod_sel]
        break

    except IndexError:
        print("You must define module ref data to use this module ...")
        break

In [ ]:
# Put relevant data into a dict for easy use
# ignore any other columns that may be database specific
# as they aren't needed

ref = dict(
  # module_id  =  ref_data['index'].values[0],
    i_sc = ref_data['i_sc'].values[0],
    i_mp = ref_data['i_mp'].values[0],
    v_mp = ref_data['v_mp'].values[0],
    v_oc = ref_data['v_oc'].values[0],

    alpha_i_sc = ref_data['alpha_i_sc'].values[0],
    beta_v_oc = ref_data['beta_v_oc'].values[0],
    alpha_i_mp = ref_data['alpha_i_mp'].values[0],
    beta_v_mp = ref_data['beta_v_mp'].values[0],
    gamma_p_mp = ref_data['gamma_p_mp'].values[0],
)

# create p_mp and ff in case they don't exist
ref['p_mp'] = ref['i_mp'] * ref['v_mp']

ref['ff'] = ref['p_mp'] / (ref['i_sc'] * ref['v_oc'])

## Import measured data (outdoor or matrix)

DateTime, Met and Raw module measurements.  

### Read in selected measured file data 

In [ ]:
meas = pd.read_csv(
    root_dir + '\\mlfm_data\\meas_gtw\\' + mlfm_meas_file,
    index_col='date_time'
)

### Calculate useful data columns for meas

In [ ]:
# normalise poa_global to kW/m^2
meas['poa_global_kwm2'] = meas['poa_global'] / 1000

# calculate p_mp as it might be missing
meas['p_mp'] = meas['i_mp'] * meas['v_mp']

#show some meas data
meas.head(3)

### Count how many mlfm variables are in the data  
usually matrix=4 (i_sc,i_mp,v_mp, v_oc)  
and iv=6 (i_sc,i_mp,v_mp, v_oc + r_sc, r_oc) 

In [ ]:
def get_qty_mlfm_vars(dmeas):
    '''
    Find the quantity of MLFM variables in the measured data
    (e.g. I_MP+V_MP=2, MATRIX=4, IV_CURVE=6).
    
    Parameters
    ----------
    dmeas : dataframe
        measured weather data
        'poa_global', 'temp_module', 'wind_speed'
        and measured electrical/thermal values
        'i_sc' .. 'v_oc', temp_module.
    
    Returns
    -------
    qty_mlfm_vars : int
        number of mlfm_values present in data usually
        2 = (imp, vmp) from mpp tracker
        4 = (i_sc, i_mp, v_mp, v_oc) from matrix
        6 = (i_sc, i_mp, v_mp, v_oc, r_sc, r_oc) from iv curve.
    '''
    # find how many mlfm variables were measured
    qty_mlfm_vars = 0
    for mlfm_sel in ('i_sc', 'r_sc', 'i_mp','v_mp', 'r_oc', 'v_oc'):
        if mlfm_sel in dmeas.columns:
            qty_mlfm_vars += 1
            #print(qty_mlfm_vars, mlfm_sel)
    
    return qty_mlfm_vars

In [ ]:
qty_mlfm_vars = get_qty_mlfm_vars(meas)

## Normalise MLFM values norm from meas and ref dataframes  

Fig 2 illustrates the loss factors model (LFM).  
It uses the shape and values from dc measurements to quantify the values of each  
of the loss factors going from (1) ref_p_mp to (5) meas_p_mp.  

Fig 2: Loss Factors Model :  
![lfm_0_4.png](mlfm_data/figs/mlfm_iv.png)  

1) ref_p_mp = Initial datasheet value at STC.

2) Multiply by 1/FF to get to (ref_i_sc * ref_v_oc) to start to analyse current and voltage losses  

3) Three 'current' losses get from ref_i_sc to norm_i_mp
   - norm_i_sc = measured / expected isc (purple)
   - norm_r_sc = loss caused by 'shunt resistance' slope at i_sc (orange)
   - norm_i_ff = loss caused by 'current part' of fill factor (green).  
    
    
4) Three 'voltage' losses (plus a temperature coefficient) get from from ref_v_oc to norm_v_mp     
   - norm_v_ff = loss caused by 'voltage part' of fill factor (blue)
   - norm_r_oc = loss caused by 'series resistance' slope at v_oc (pink)
   - norm_v_oc_t = measured / expected v_oc temp_corrected (brown)
   - norm_temp_corr = optional temp correction subtracted from v_oc (red).    
    
    
5) These losses cause the performance to fall to pr_dc = meas_p_mp / ref_p_mp  

pr_dc = 1/ff *                                    
    (norm_i_sc * norm_r_sc * norm_i_ff ) *             
    (norm_v_ffv * norm_r_oc * norm_v_oc_t * norm_temp_corr )   

Note:  
The gamma temperature correction is subtracted from voc for simplicity.  
In reality there will be temperature dependencies for i_sc and ff but they are smaller.

In [ ]:
norm = mlfm_meas_to_norm(meas, ref, qty_mlfm_vars)

# show some normalised data
norm.head(3)

### Make irradiance and temperature bins for pivot tables 
(Gbin=100W/m^2, Tbin=5C)

In [ ]:
# poa_global bin = 100W/m2
norm['poa_global_bin'] = meas['poa_global'].round(-2)

# temp_module bin = 5C
norm['temp_module_bin'] = (5 * round(meas['temp_module'] / 5,0))

## Perform sanity checks on meas and norm data  

It's easier to sanity check and study normalised data than raw values.  
1) Remove bad, missing, unwanted  or outlier data  
2) User defined limits may depend on data scatter and degradation   
3) Can either select on values e.g. '0.5 <pr_dc < 1.5' or on '>x stdev from mean'
4) Possible to select on dates if desired.  

In [ ]:
# select by irradiance poa_global range e.g. 100-1100W/m2
meas = meas[(meas['poa_global'] >= 100) &
            (meas['poa_global'] <= 1100)]

# if there's date_time can select by it, i.e. not matrix data
### better if index is formatted as a date

# if qty_mlfm_vars == 6:

    # not for matrix as they don't contain dates
    # example
    # meas = meas[(meas.index > '2016-01-01') &
    #        (meas.index < '2017-01-01')]


In [ ]:
# remove specific mlfm values outside limits e.g. <0.5 or >1.5
norm = norm[((norm['pr_dc'] > 0.5) & 
             (norm['pr_dc'] < 1.5))]

# remove all mlfm values outside x~3 stdevs
if qty_mlfm_vars == 6:
    # only needed for outdoor data as indoor ought to be less scattered
    # remove all mlfm data > x stdev usually 3
    stdevs = 3

    for lfm in ('i_sc', 'r_sc', 'i_ff', 'v_ff', 'r_oc','v_oc'):
        norm = norm[
            ((norm[lfm] - norm[lfm].mean()) /
                 norm[lfm].std()).abs() < stdevs
        ]

### Filter only matching rows from meas and norm data
like an inner join but leave data in separate norm and meas frames

In [ ]:
#drop meas rows that aren't in norm
meas_not_in_norm = ~meas.index.isin(norm.index)
meas = meas.drop(meas[meas_not_in_norm].index)

#drop norm rows that aren't in meas
norm_not_in_meas = ~norm.index.isin(meas.index)
norm = norm.drop(norm[norm_not_in_meas].index)

## Plot normalised MLFM data vs irradiance  

For outdoor data -  
LFM values norm() should be narrow, smooth lines (around 70-120% on the yaxis).

For matrix data -  
LFM values norm() should be close, almost parallel lines (around 70-120% on the yaxis).

1. Higher values are always better (unlike measured values such as  
    Rseries or Io where lower is better)
1. Accurate measurements and a stable module result in narrowest lines  
1. v_oc and r_sc tend to fall at low light levels ( / left)  
1. r_oc tends to fall at high light levels ( \ right)  
1. i_ff and v_ff are usually fairly flat ( - ) 
1. i_sc may vary the most due to spectral sensitivity, soiling, shading  
    and/or snow (if not properly corrected).    

## Fig 3 : Normalised mlfm values vs. irradiance.

All traces should be thin, usually around 0.9 ± 0.1  
i_sc may be more scattered if there is uncorrected soiling, spectral and angle of incidence ###

In [ ]:
# scatter plot normalised values vs. irradiance
plot_mlfm_scatter(meas, norm, mlfm_meas_file, qty_mlfm_vars)

## Convert multiplicative to subtractive losses to show on a stack plot  

LFM losses can be analysed as either    

- multiplicative    
pr_dc = 1/ff * ( norm(i_sc) * norm(r_sc) * norm(i_ff) * 
        norm(v_ff) * norm(r_oc) * norm(v_oc_t) * norm(temp_corr) ).  
 
   
  
-  subtractive  
pr_dc = 1/ff -   (stack(i_sc) + stack(r_sc) + stack(i_ff) -  
        stack(v_ff) + stack(r_oc) + stack(v_oc_t) + stack(temp_corr) ). 
  
Multiplicative losses are easier to understand but to represent them on a graph  
it's easier to show them as a stacked plot where the values are 'translated'  
so the sum of the stacked losses is shown to equate to the product of the  
multiplicative losses.

In [ ]:
# translate multiplicative to stack losses and add to dataframe df
stack = mlfm_norm_to_stack(norm, ref, qty_mlfm_vars)

# show some stack losses
stack.head(3)

## Plot stack losses vs. measurement  

Fig 4 Shows how to quantify losses by loss parameters stack(i_sc, .. v_oc).  

Fig 4 Stacked losses by measurement  
![stack5D_0_4.png](mlfm_data/figs/mlfm_stack.png)  

- It plots them in a stacked format from the lossless limit 1/ff (top)  
  subtracting each loss value in turn until it reaches pr_dc (bottom).  
  
- This figure shows a typical c-Si module for four clear days for  
  different months July to Oct in AZ.   
  
- In the middle of the days the high irradiance results in the biggest  
  losses being due to r_oc (red, ~rseries, pink) and temp_module  
  (as the module heats to 60C).   
  
- Early mornings/late afternoons there is a slight Isc gain (purple,  
  top, due to spectral mismatch) but an Isc loss mid day due to soiling.  

Stack losses are indicated by their colours  
(from top to bottom for mlfm4=matrix and mlfm6=ivcurve)   

`+-----+----+-------+--------+------------+--------------+`  
`|  1  | 2  |   4   |   6    | <qty_mlfm  |              |`  
`|     |mpp | matrix|iv_curve| colours    |    comments  |`  
`| 1/FF+----+-------+--------+------------+--------------+`  
`| v   |    | (i_sc)| (i_sc) | (purple)   | (optional)   |`  
`| v   |    |- - - -|- - - - |- - - - - - |- - - - - - - |`  
`| v   |    |       |  r_sc  | orange     |  current     |`  
`| v   |    +       +--------+------------+              |`  
`| v   |i_mp|  i_mp |  i_ff  | green      |  losses      |`  
`|-v---|----+-------+--------+-black------+--------------+`   
`| v   |v_mp|  v_mp |  v_ff  | blue       |  voltage     |`  
`| v   |    +       +--------+------------+              |`  
`| v   |    |       |  r_oc  | pink       |  losses      |`  
`| v   |    +-------+--------+------------+              |`  
`| v   |    |  v_oc |  v_oc  | brown      |              |`  
`| v   |    |- - - -|- - - - |- - - - - - |- - - - - - - |`   
`| v   |    |(temp  |(temp   | (red)      | (optional)   |`  
`| v   |    | corr) | corr)  |            |              |`  
`|pr_dc+----+-------+--------+------------+--------------+`  

Graph options :  

is_i_sc_self_ref : boolean   
    = self corrects i_sc to remove angle of incidence, spectrum, 
    snow or soiling.  
    
is_v_oc_temp_module_corr : boolean  
    = calc temperature loss due to gamma, subtract from voc loss   

### Fig 5 : stacked loss values vs. date and time or matrix measurement

In [ ]:
# plot stack loss vs. time (or measurement) chart
plot_mlfm_stack(
    dmeas=meas,
    dnorm=norm,
    dstack=stack,                   # dataframe measurements
    ref=ref,                        # dataframe reference STC
    mlfm_file_name=mlfm_meas_file,  # name of data file
    qty_mlfm_vars=qty_mlfm_vars,    # number of mlfm measurements usually 4 or 6
    xaxis_labels=12,                # show this many x_labels or 0 to show all
    is_i_sc_self_ref=False,         # is isc self referenced?
    is_v_oc_temp_module_corr=True,  # is voc temperature corrected?
)

## Fit mechanistic model to measured weather and normalised losses  

Perform a Mechanistic Performance Model (MPM) fit to the mlfm parameters  
poa_global (kW/m$^2$), temp_module (C), wind_speed (ms$^-$$^1$).  

MPM_6 = c_1 + c_2 * (temp_module-25) + c_3 * log10(poa_global_kwm2) +   
        c_4 * poa_global_kwm2 + c_5 * wind_speed + c_6 / poa_global_kwm2  


Report the fit (coeffs) and error (errs) coefficients.   

In [ ]:
# choose which no0rmalised mlfm parameter to model e.g. pr_dc or i_sc..v_oc  
mlfm_sel = 'pr_dc' 

# FIX THIS WARNING,
# SettingWithCopyWarning:
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
# TRY TO DO A BETTER METHOD THAN JUST HIDING IT

norm, cc, ee,  coeffs, errs = mlfm_fit(meas, norm, mlfm_sel) # qty_mlfm_vars)

# Fix a bug with fit routine which gives a
# finite cc[4] even if all the ws data is 0
# this won't matter until cc is applied to other
# data with some ws <>0 when it will give bad results
if np.mean(meas.wind_speed) == 0:
    cc[4] = 0
    c_5 = 0


## Show residual fit vs. measured for MLFM parameter  

In [ ]:
def plot_residuals(dmeas, dnorm, fit, title):
    ''' 
    Scatter plot residuals to normalised measured

    Parameters
    ----------
    dmeas : dataframe
        measured weather data
        'poa_global', 'temp_module', 'wind_speed'
        and measured electrical/thermal values
        'i_sc' .. 'v_oc', temp_module.
    
    
    dnorm : dataframe
        normalised multiplicative lfm loss values 'i_sc' .. 'v_oc'
        where pr_dc = 1/ff * product('i_sc', ... 'v_oc').
        
    fit : string
        name of fitted variable e.g. 'pr_dc'.
        
    title : string
        title for graph e.g. mlfm_meas_file.
    '''
    
    fig, ax1 = plt.subplots()

    plt.title(title)

    plt.ylabel('fit ' + fit + ' * poa_global_kwm2')
    ax1.set_ylim(0, 1.2)

    plt.xlabel('meas ' + fit + '* poa_global_kwm2')
    ax1.set_xlim(0, 1.2)

    plt.plot(
        dnorm[fit] * dmeas['poa_global_kwm2'],
        dnorm['calc_' + fit] * dmeas['poa_global_kwm2'],
        'c^',
        label = fit
    )

    # plot 1:1 line to show optimum fit
    plt.plot((0,1.2),(0,1.2), 'ko-')

    plt.legend(loc='upper left')
    plt.show()

### Fig 6 : fit_mlfm_sel * poa_global vs. measured_mlfm_sel * poa_global

In [ ]:
# plot fit vs. measured, include a 1:1 line for comparison
fit_plot = plot_residuals(meas, norm, mlfm_sel, 'residual ' + mlfm_meas_file)

## Plot heatmap of mean residual vs. tenp_module and poa_global

Show a heatmap of the average residual (meas - fit) error  
for each irradiance (100W/m^2)  and tmod bin (5C).

In [ ]:
def plot_heatmap(dmeas, dnorm, fit, y_axis, x_axis, z_axis, title):
    '''
    Plot a heatmap of Z vs. binned X and Y axes.

    Parameters
    ----------
    dmeas : dataframe
        measured weather data
        'poa_global', 'temp_module', 'wind_speed'
        and measured electrical/thermal values
        'i_sc' .. 'v_oc', temp_module.

    dnorm : dataframe
        normalised multiplicative lfm loss values 'i_sc' .. 'v_oc'
        where pr_dc = 1/ff * product('i_sc', ... 'v_oc').

    fit : string
        fitted parameter e.g. 'pr_dc'.

    x_axis : string
        binned x axis e.g. 'poa_global_bin'.

    y_axis : string
        binned y axis e.g. 'temp_module_bin'.

    z_axis : string
        value as a colour surface plot e.f. 'diff_pr_dc'.

    title : string
        title for graph e.g. mlfm_meas_file.

    '''

    df_piv = pd.pivot_table(
        dnorm,
        index=y_axis,       # e.g. 'temp_module_bin'
        columns=x_axis,     # e.g. 'poa_global_bin'
        values=z_axis,      # value to aggregate
        fill_value=0,       # fill empty cells with this ?
        aggfunc=[np.mean],  # e.g. min, np.sum, len->count
        margins=False,      # grand totals hide
        dropna=True         # hide missing rows or columns
    )

    fig, ax1 = plt.subplots()

    # force z limits to be -2% to +2% if desired
    df_piv = df_piv.clip(lower = -0.02, upper = +0.02)

    im = ax1.imshow(
        df_piv,
        cmap='RdYlBu',
        origin='lower'
    )

    cbar = ax1.figure.colorbar(im, ax=ax1, shrink=0.75, label=z_axis)

    #Y AXIS : show only 1 of each y_skip labels
    y_ticks = df_piv.shape[0]
    plt.yticks(np.arange(0, y_ticks), rotation=0)
    yax2 = [''] * y_ticks
    y_skip = 2
    y_count = 0
    while y_count < y_ticks:
        if y_count % y_skip == 0:
            yax2[y_count] = df_piv.index[y_count]
        y_count += 1

    ax1.set_yticklabels(yax2)
    ax1.set_ylabel(y_axis)

    # X AXIS : show only 1 of each x_skip labels
    x_ticks = df_piv.shape[1]
    plt.xticks(np.arange(0, x_ticks), rotation=90)

    xax2 = [''] * x_ticks
    x_skip = 2
    x_count = 0
    while x_count < x_ticks:
        if x_count % x_skip == 0:
            xax2[x_count] = df_piv.columns.levels[1][x_count]
        x_count += 1
        
    ax1.set_title(title)

    ax1.set_xticklabels(xax2)
    ax1.set_xlabel(x_axis)

    ax1.grid( color='k', linestyle=':', linewidth=1)

### Fig 7 : Residual MLFM fit heatmap vs. poa_global and temp_module.

In [ ]:
# plot heatmap
heatmap_plot = plot_heatmap(
    dnorm=norm,
    dmeas=meas,
    fit=mlfm_sel,
    y_axis='temp_module_bin',
    x_axis='poa_global_bin',
    z_axis='diff_' + mlfm_sel,
    title='residual ' + mlfm_meas_file
)

## Read in complete (G,T) Matrix to fill with MLFM predicted values  

Read in a matrix with complete values of   
Irradiance (G=100,200 .. 1100,1200) and module temperature (T=0,5 .. 65,70)  
to predict all MPM values 

In [ ]:
# read in the complete matrix data
matr = pd.read_csv(root_dir + '\\mlfm_data\\ref\\mlfm_matrix.csv', index_col = 'id')

matr['poa_global_kwm2'] = matr['poa_global'] / 1000

matr.columns

## Predict performance from MPM fit coefficients   

1. generate predicted mpm data  
2. create a pivot table mpm(g,t)  
3. show as a heat map

In [ ]:
# populate pivot table from predicted mpm data
matr[mlfm_sel] = mlfm_6(matr, cc[0], cc[1], cc[2], cc[3], cc[4], cc[5])

## Plot heatmap of predicted MLFM values vs. temp_mod and poa_global bins

In [ ]:
def plot_contourf(df, x_axis, y_axis, z_axis, title,
                  vmin=0, vmax=1.2, levels=5):
    ''' 
    Plot filled contour plot Z vs. X and Y bins.

    Parameters
    ----------
    df : dataframe
        measured or noralised data containing weather columns
        (poa_global, temp_module and wind_speed).

    x_axis : string
        binned x axis e.g. 'poa_global'.

    y_axis : string
        binned y axis e.g. 'temp_module'.

    z_axis : string
        measured value as a colour surface plot.

    title : string
        title for graph e.g. mlfm_meas_file.

    vmin, vmax : float
        minimum and maximum values for contour chart ###

    '''  
   
    piv = pd.pivot_table(
        df,
        index=y_axis,
        columns=x_axis,
        values=z_axis,
        fill_value=0,      # fill empty cells?
        aggfunc=[np.mean], # min, np.sum, len->count
        margins=False,     # grand totals
        dropna=True        # hide missing rows or columns
    )

    piv = piv.clip(vmin, vmax)

    fig, ax1 = plt.subplots()

    cs = plt.contourf(
        piv,
        cmap='RdYlBu',  # or 'nipy_spectral',
      # origin='lower'
      # nchunkint=1,
        levels=levels,
        vmin=vmin,
        vmax=vmax
    )

    cbar = fig.colorbar(cs, ax=ax1)
    cbar.ax.set_ylabel(z_axis,
                       rotation=90,
                       va='bottom',
                       labelpad=+30)

    plt.title(title)

    y_ticks = piv.shape[0]
    plt.yticks(np.arange(0, y_ticks), rotation=0)

    # show only 1 of each y_skip labels
    yax2 = [''] * y_ticks
    y_skip = 2
    y_count = 0
    while y_count < y_ticks:
        if y_count % y_skip == 0:
            yax2[y_count] = piv.index[y_count]
        y_count += 1

    ax1.set_yticklabels(yax2)
    ax1.set_ylabel(y_axis)

    x_ticks = piv.shape[1]
    plt.xticks(np.arange(0, x_ticks), rotation=90)

    # show only 1 of each x_skip labels
    xax2 = [''] * x_ticks
    x_skip = 2
    x_count = 0
    while x_count < x_ticks:
        if x_count % x_skip == 0:
            xax2[x_count] = piv.columns.levels[1][x_count]
        x_count += 1

    ax1.set_xticklabels(xax2)
    ax1.set_xlabel(x_axis)

    ax1.grid( color='k', linestyle=':', linewidth=1)

### Fig 8 : Contour plot of predicted mlfm_sel + vs. poa_global and temp_mod.

In [ ]:
contour_plot = plot_contourf(
    df=matr,
    y_axis='temp_module',
    x_axis='poa_global',
    z_axis=mlfm_sel,
    title='matrix predicted ' + mlfm_meas_file,
    vmin=0.6,
    vmax=1.05,
    levels=9
)

### Fig 9 : Contour plot of measured mlfm_sel  vs. poa_global and temp_mod.

In [ ]:
contour_plot = plot_contourf(
    df=norm,
    y_axis='temp_module_bin',
    x_axis='poa_global_bin',
    z_axis=mlfm_sel,
    title='avg normalised ' + mlfm_meas_file,
    vmin=0.6,
    vmax=1.05,
    levels=9
)

## References  
     
The Loss Factors Model (LFM) and Mechanistic Performance Model (MPM)  
together known as "MLFM" have been developed by SRCL and Gantner Instruments  
(previously Oerlikon Solar and Tel Solar) since 2011 MLFM and 2017 MPM  
  
.. [1] J. Sutterlueti(now Gantner Instruments) and  S. Ransome  
 '4AV.2.41 Characterising PV Modules under Outdoor Conditions:  
What's Most Important for Energy Yield'  
26th EU PVSEC 8 September 2011; Hamburg, Germany  
http://www.steveransome.com/pubs/2011Hamburg_4AV2_41.pdf  

.. [2] Steve Ransome and Juergen Sutterlueti(Gantner Instruments)  
  'Choosing the best Empirical Model for predicting energy yield'   
  7th PV Energy Rating and Module Performance Modeling Workshop,  
  Canobbio, Switzerland 30-31 March, 2017  

.. [3] S. Ransome and J. Sutterlueti (Gantner Instruments)  
'Checking the new IEC 61853.1-4 with high quality 3rd party data to  
benchmark its practical relevance in energy yield prediction'  
PVSC June 2019 Chicago, USA  
http://www.steveransome.com/PUBS/1906_PVSC46_Chicago_Ransome.pdf

.. [4] Steve Ransome (SRCL) and Juergen Sutterlueti (Gantner Instruments)  
'5CV.4.35 Quantifying Long Term PV Performance and Degradation  
under Real Outdoor and IEC 61853 Test Conditions  
Using High Quality Module IV Measurements'   
36th EU PVSEC Sep 2019  
http://www.steveransome.com/PUBS/1909_5CV4_35_PVSEC36_Marseille_Ransome_PPT.pdf

.. [5] Steve Ransome (SRCL)  
'How to use the Loss Factors and Mechanistic Performance Models  
effectively with PVPMC/PVLIB'   
PVPMC Webinar on PV Performance Modeling Methods, Aug 2020  
https://pvpmc.sandia.gov/download/7879/  

.. [6] W.Marion et al (NREL)  
'New Data Set for Validating PV Module Performance Models'  
https://www.researchgate.net/publication/286746041_New_data_set_for_validating_PV_module_performance_models  
https://www.nrel.gov/docs/fy14osti/61610.pdf

Many more papers are available at www.steveransome.com  
